In [1]:
include("../../../julia/FNC_init.jl")

  Activating 

project at `~/Documents/GitHub/fnc`


[**Demo %s**](#demo-condition-bound)


```{index} ! Julia; cond
```
Julia has a function `cond` to compute matrix condition numbers. By default, the 2-norm is used. As an example, the family of *Hilbert matrices* is famously badly conditioned. Here is the $6\times 6$  case.
```{tip}
:class: dropdown
Type `\kappa` followed by <kbd>Tab</kbd> to get the Greek letter $\kappa$.
```

In [2]:
A = [ 1 / (i + j) for i in 1:6, j in 1:6 ]
κ = cond(A)

5.109816297946132e7

Because $\kappa\approx 10^8$, it's possible to lose nearly 8 digits of accuracy in the process of passing from $\mathbf{A}$ and $\mathbf{b}$ to $\mathbf{x}$. That fact is independent of the algorithm; it's inevitable once the data are expressed in finite precision. 

Let's engineer a linear system problem to observe the effect of a perturbation. We will make sure we know the exact answer.

In [3]:
x = 1:6
b = A * x

6-element Vector{Float64}:
 4.407142857142857
 3.564285714285714
 3.013095238095238
 2.6174603174603175
 2.317279942279942
 2.0807359307359308

Now we perturb the system matrix and vector randomly by $10^{-10}$ in norm.

In [4]:
# type \Delta then Tab to get Δ
ΔA = randn(size(A));  ΔA = 1e-10 * (ΔA / opnorm(ΔA));
Δb = randn(size(b));  Δb = 1e-10 * normalize(Δb);

We solve the perturbed problem using pivoted LU and see how the solution was changed.

In [5]:
new_x = ((A + ΔA) \ (b + Δb))
Δx = new_x - x

6-element Vector{Float64}:
 -1.6599759181112894e-5
  0.00029415492849960145
 -0.001575236610755315
  0.003546974548783055
 -0.0035446614923344555
  0.001298074522693149

Here is the relative error in the solution.

In [6]:
@show relative_error = norm(Δx) / norm(x);

relative_error = norm(Δx) / norm(x) = 0.0005683871926046626

And here are upper bounds predicted using the condition number of the original matrix.

In [7]:
println("Upper bound due to b: $(κ * norm(Δb) / norm(b))")
println("Upper bound due to A: $(κ * opnorm(ΔA) / opnorm(A))")

Upper bound due to b: 0.000672366771437133


Upper bound due to A: 0.004566989873939065


Even if we didn't make any manual perturbations to the data, machine roundoff does so at the relative level of $\macheps$.

In [8]:
Δx = A\b - x
@show relative_error = norm(Δx) / norm(x);
@show rounding_bound = κ * eps();

relative_error = norm(Δx) / norm(x) = 7.822650774976615e-10
rounding_bound = κ * eps() = 1.134607141116935e-8


Larger Hilbert matrices are even more poorly conditioned:

In [9]:
A = [ 1 / (i + j) for i=1:14, j=1:14 ];
κ = cond(A)

5.802584125151949e17

Note that $\kappa$ exceeds $1/\macheps$. In principle we therefore may end up with an answer that has relative error greater than 100%.

In [10]:
rounding_bound = κ*eps()

128.8432499613623

Let's put that prediction to the test.

In [11]:
x = 1:14
b = A * x  
Δx = A\b - x
@show relative_error = norm(Δx) / norm(x);

relative_error = norm(Δx) / norm(x) = 4.469466154206132


As anticipated, the solution has zero accurate digits in the 2-norm.